In [12]:
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy.engine import URL, create_engine, Inspector
 
# Use your own server connection here

server = 'localhost'
database = 'AIFMRM_ERS'
username = 'sa'
password = 'pohtoj-Jewtix-vudko9'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

df = pd.read_sql_query('SELECT * FROM dbo.tbl_BA_Beta_Output', cnxn)

print(df)
print(type(df))

cnxn_url = URL.create(
    'mssql+pyodbc',
    database='AIFMRM_ERS',
    username= 'sa',
    password='pohtoj-Jewtix-vudko9',
    query=dict(driver='ODBC Driver 17 for SQL Server'))

engine = create_engine(cnxn_url)


# initialise connection via context manager           
with engine.connect() as cnxn:
        tables_df = pd.read_sql('SELECT [name] AS [table_name] FROM sys.tables', cnxn)
        table_name_list = tables_df.table_name
        select_template = 'SELECT * FROM {table_name}'
        # Dictionary of table names and their respective SQL queries
        frames_dict = {}
        for tname in table_name_list:
                query = select_template.format(table_name = tname)
                frames_dict[tname] = pd.read_sql(query, cnxn)
tbl_Industry_Classification_Benchmark = frames_dict["tbl_Industry_Classification_Benchmark"]
tbl_Index_Constituents = frames_dict["tbl_Index_Constituents"]
tbl_BA_Beta_Output = frames_dict["tbl_BA_Beta_Output"]
tbl_FTSEJSE_Index_Series = frames_dict["tbl_FTSEJSE_Index_Series"]
tbl_EOD_Equity_Data = frames_dict["tbl_EOD_Equity_Data"]
tbl_EOD_Interest_Rate_Data = frames_dict["tbl_EOD_Interest_Rate_Data"]
tbl_Beta_Output = frames_dict["tbl_Beta_Output"]     

/usr/local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


            Date Instrument Index Start Date   End Date  % Days Traded  \
0     2017-09-29        ACE  J200 2012-09-28 2017-09-27       0.534455   
1     2017-09-29        ACE  J203 2012-09-28 2017-09-27       0.534455   
2     2017-09-29        ACE  J250 2012-09-28 2017-09-27       0.534455   
3     2017-09-29        ACE  J257 2012-09-28 2017-09-27       0.534455   
4     2017-09-29        ACE  J258 2012-09-28 2017-09-27       0.534455   
...          ...        ...   ...        ...        ...            ...   
32795 2021-03-31        ZED  J200 2016-03-31 2021-03-31       1.000000   
32796 2021-03-31        ZED  J203 2016-03-31 2021-03-31       1.000000   
32797 2021-03-31        ZED  J250 2016-03-31 2021-03-31       1.000000   
32798 2021-03-31        ZED  J257 2016-03-31 2021-03-31       1.000000   
32799 2021-03-31        ZED  J258 2016-03-31 2021-03-31       1.000000   

       Data Points     Alpha      Beta  SE Alpha   SE Beta  p-Value Alpha  \
0               60  0.004147  0.31

Function 1

In [7]:
def GetICsAndWeights(rDate,IndexCode,dbo_tbl_Index_Constituents):
    #Store tbl_Index_Constituents from frames_dict
    tbl_Index_Constituents = frames_dict[dbo_tbl_Index_Constituents]

    #rDate will be supplied by the user: consisting of year and Quarter 
    rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m-%d")
    rDate_Quarter = rDate.quarter
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = tbl_Index_Constituents["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Quarter = Dates_Col.quarter
    Dates_Col_Year = Dates_Col.year

    #Filter tbl_Index_Constituents using supplied quarter and year data from rData
    tbl_Index_Constituents_Date = tbl_Index_Constituents.loc[(Dates_Col_Quarter == rDate_Quarter) & (Dates_Col_Year == rDate_Year),]


    #IndexCode is provided by user: "ALSI", "FLED", "LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY" or "ALTI"
    IndexCode = IndexCode #provided as input by the user. "ALSI" for testing purposes

    #function to identify The index column that must be searched
    def Index_Col_Identifier(argument):
        match argument:
            case "FLED":
                return "ALSI New"
            case  "LRGG"|"MIDC"|"SMILC":
                return "Index New"
            case default:
                return argument+" New"
    
    IndexCode_Col = Index_Col_Identifier(IndexCode) #Obtain column name to search relevant rows

    #Filter tbl_Index_Constituents_Date using supplied IndexCode in the column identified from Index_Col_Identifier function
    tbl_Index_Constituents_final = tbl_Index_Constituents_Date[tbl_Index_Constituents_Date[IndexCode_Col] == IndexCode]

    #Generate results table with Shares and corresponding share weights
    Alpha = pd.DataFrame(tbl_Index_Constituents_final.loc[:,"Alpha"])
    Gross_Market_Capitalisation = np.array(tbl_Index_Constituents_final.loc[:,"Gross Market Capitalisation"])
    Weigths = pd.DataFrame(Gross_Market_Capitalisation/np.sum(Gross_Market_Capitalisation))
    Results = pd.concat([Alpha.reset_index(drop=True), Weigths.reset_index(drop=True)],axis=1)
    Results.columns = ['Alpha','Weights']

    return Results


rDate = '2019-6-05'
IndexCode = "ALSI"
dbo_tbl_Index_Constituents = "tbl_Index_Constituents"
Output1 = GetICsAndWeights(rDate,IndexCode,dbo_tbl_Index_Constituents)

Function 2

In [8]:
def GetBetasMktAndSpecVols(rDate,ICs,dbo_tbl_BA_Beta_Output,mktIndexCode):

    #Store tbl_BA_Beta_Output from frames_dict
    tbl_BA_Beta_Output = frames_dict[dbo_tbl_BA_Beta_Output]

    #rDate will be supplied by the user: consisting of year and Quarter 
    rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m-%d")
    rDate_Quarter = rDate.quarter
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = tbl_BA_Beta_Output["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Quarter = Dates_Col.quarter
    Dates_Col_Year = Dates_Col.year

    #Filter tbl_BA_Beta_Output using supplied quarter and year data from rData
    tbl_BA_Beta_Output_Date = tbl_BA_Beta_Output.loc[(Dates_Col_Quarter == rDate_Quarter) & (Dates_Col_Year == rDate_Year),]
    #return tbl_BA_Beta_Output_Date
    #Market index code provided by the user which could be "J203", "J200", "J250", "J257" or "J258"
    mktIndexCode = mktIndexCode

    #Filter tbl_BA_Beta_Output_Dates using provided mktIndexCode
    tbl_BA_Beta_Output_mktIndex = tbl_BA_Beta_Output_Date.loc[tbl_BA_Beta_Output_Date["Index"] == mktIndexCode]
    #return tbl_BA_Beta_Output_mktIndex

    #list of IndexCodes obtain from  the Alpha column in the Output of Function 1.
    ICs = ICs
    tbl_BA_Beta_Output_IC = tbl_BA_Beta_Output_mktIndex.loc[tbl_BA_Beta_Output_mktIndex["Instrument"].isin(ICs)]

    #Generate results table with Shares and corresponding share weights
    Betas = tbl_BA_Beta_Output_IC.loc[:,"Beta"]
    specVols = tbl_BA_Beta_Output_IC.loc[:,"Unique Risk"]
    mktVol = tbl_BA_Beta_Output_IC.loc[:,"Total Risk"]
    Results = pd.concat([Betas.reset_index(drop=True), specVols.reset_index(drop=True),mktVol.reset_index(drop=True)],axis=1)
    Results.columns = ['Betas','specVols','mktVol']

    return Results
    #return tbl_BA_Beta_Output_IC

ICs = Output1["Alpha"]
dbo_tbl_BA_Beta_Output = "tbl_BA_Beta_Output"
mktIndexCode = "J203"
Output2 = GetBetasMktAndSpecVols(rDate,ICs,dbo_tbl_BA_Beta_Output,mktIndexCode)

Function 3

In [9]:
def CalcStats(weights,betas,specVols,mktVol):

    weights = np.transpose(np.matrix(Output1["Weights"]))
    betas = np.transpose(np.matrix(Output2["Betas"]))
    specVols = np.transpose(np.matrix(Output2["specVols"]))
    mktVol = np.array(Output2["mktVol"])
    mktVol = np.average(mktVol) #Assuming the mktVol is the average. NEED TO CONFIRM

    pfBeta = np.matmul(np.transpose(weights),betas)

    sysCov = np.matmul(betas,np.transpose(betas))*(mktVol**2)

    pfSysVol = np.transpose(weights)@betas@np.transpose(betas)@(weights)*(mktVol**2)

    specCov = np.diagflat(specVols)@np.diagflat(specVols)

    pfSpecVol = np.transpose(weights)@(specCov)@weights

    totCov = sysCov + specCov

    pfVol = pfSysVol + pfSpecVol

## synthetic table


In [13]:
from unittest import result

''' 
#rDate will be supplied by the user: consisting of year and Quarter 
     rDate = rDate
    rDate = pd.to_datetime(rDate, format = "%Y-%m")
    rDate_Month = rDate.month
    rDate_Year = rDate.year

    #search tbl_Index_Constituents Date column and find Quarter and Year for each date in column
    Dates_Col = dbo_tbl_Index_Constituents["Date"]
    Dates_Col = pd.arrays.DatetimeArray(Dates_Col)
    Dates_Col_Month = Dates_Col.month
    Dates_Col_Year = Dates_Col.year

 tbl_Index_Constituents_Date = dbo_tbl_Index_Constituents.loc[(Dates_Col_Month == rDate_Month) & (Dates_Col_Year == rDate_Year),]

'''
Quarter_month = {1:3, 2:6, 3:9, 4:12}

Date_1_year = '2018'
Date_1_quarter = 1
Date_1_month = str(Quarter_month[Date_1_quarter])

Date_2_year = '2018'
Date_2_quarter = 4
Date_2_month = str(Quarter_month[Date_2_quarter])

Date_1 = Date_1_year +"-"+ Date_1_month+"-01"
Date_2 = Date_2_year +"-"+ Date_2_month+"-28"

Date_1 = pd.to_datetime(Date_1, format = "%Y-%m-%d")
Date_2 = pd.to_datetime(Date_2, format = "%Y-%m-%d")

Dates = (tbl_Index_Constituents.loc[(tbl_Index_Constituents["Date"] >= Date_1) & (tbl_Index_Constituents["Date"] <= Date_2)]).loc[:,"Date"].unique() #get list of quarters between two dates
Industry_codes = tbl_Industry_Classification_Benchmark.loc[:,"Industry Code"].unique() #find all the industry codes
Industry_names = tbl_Industry_Classification_Benchmark.loc[:,"Industry"].unique()
IndexCode_temp = "TOPI"
#IndexCode is provided by user: "ALSI", "FLED", "LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY" or "ALTI"

df_Storage = pd.DataFrame(columns=['Date', 'Industry', 'Weight', 'Beta', 'SysVol', 'SpecVar'])
#put in If Else statement for conditions where the weights is 0 which will give error
it = 0
for i in Dates:
    for t in range(len(Industry_codes)):
        it = it + 1
        print("iteration:",it)
        print("Date:",i)
        
        Subsector = (tbl_Industry_Classification_Benchmark.loc[tbl_Industry_Classification_Benchmark["Industry Code"] == Industry_codes[t]]).loc[:,"Sub-Sector Code"].unique()
         #Select index code - Provided by the user

        #run through first function
        Results_1 = GetICsAndWeights(i,IndexCode_temp,tbl_Index_Constituents)
        Results_1_filtered = Results_1.loc[Results_1["ICB Sub-Sector"].isin(Subsector)] 

        #Place if-Else statement here:
        if len(Results_1_filtered)>0:

            Filtered_GMC = np.array(Results_1_filtered.loc[:,"Gross Market Capitalisation"])
            Total_GMC = np.array(Results_1.loc[:,"Gross Market Capitalisation"])

            Industry_Weights = pd.DataFrame((Filtered_GMC/np.sum(Total_GMC)))
            Industry_Weight_tot = "{:.2%}".format(float(np.sum(Industry_Weights)))

            ICs_temp = Results_1_filtered["Alpha"]
            mktIndexCode_temp = "J203"
            Results_2 = GetBetasMktAndSpecVols(i,ICs_temp,tbl_BA_Beta_Output,mktIndexCode_temp)

            mktVol_temp = Results_2["mktVol"][0]
            betas_temp = np.transpose(np.matrix(Results_2["Betas"]))
            specVols_temp = np.transpose(np.matrix(Results_2["specVols"]))

            Results_3 = CalcStats(Industry_Weights,betas_temp,specVols_temp,mktVol_temp)

            beta_output = Results_3["pfBeta"][0]
            SysVol_output = "{:.3%}".format(float(Results_3["pfSysVol"][0]))
            SpecVar_output = round(float(Results_3["pfSpecVol"][0]),6)
            
            date_output = pd.to_datetime(i)
            date_output = str(date_output.year)+"-Q"+str(date_output.quarter)


            df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': Industry_Weight_tot, 'Beta' : Results_3["pfBeta"], 'SysVol' : SysVol_output, 'SpecVar' : SpecVar_output}, ignore_index=True)
            print("Industry:", Industry_names[t])
        else:
            weight_output = "{:.2%}".format(0.000)
            date_output = pd.to_datetime(i)
            date_output = str(date_output.year)+"-Q"+str(date_output.quarter)
            df_Storage = df_Storage.append({'Date': date_output, 'Industry': Industry_names[t], 'Weight': weight_output, 'Beta' : "     -", 'SysVol' : "     -", 'SpecVar' : "     -"}, ignore_index=True)
            print("Industry:", Industry_names[t])

iteration: 1
Date: 2018-03-19T00:00:00.000000000


TypeError: unhashable type: 'DataFrame'